# Ollama + RAG
Fonte: [RAG with LLaMA Using Ollama: A Deep Dive into Retrieval-Augmented Generation](https://medium.com/@danushidk507/rag-with-llama-using-ollama-a-deep-dive-into-retrieval-augmented-generation-c58b9a1cfcd3)

## Implementação

### Pre Requisitos

```sh
!pip install langchain
!pip install -U langchain-community
!pip install langchain-ollama
!pip install langchain-huggingface
!pip install sentence-transformers
!pip install faiss-cpu
```

### 1. Ingestão dos dados

In [1]:
from langchain.text_splitter import MarkdownHeaderTextSplitter
from pathlib import Path

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# Split the document into chunks
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
docs = text_splitter.split_text(Path("./data/historia_inedita.md").read_text())
docs

[Document(metadata={'Header 1': '**A Branca de Neve e o Chapeuzinho Vermelho: Uma Noite Mágica**', 'Header 2': 'Capítulo 1: Um Caso de Confusão'}, page_content='Em uma vila encantada, onde a luz do sol refletia nas casas de madeira e as flores brilhavam como diamantes sob as árvores, viviam Branca de Neve e seu avô. Ela era conhecida por sua pele branca como a neve e pelo sorriso radiante que iluminava seus dias.  \nUma noite, enquanto preparava um suco de morango para o avô doente, uma sombra escura se moveu na floresta ao lado. Era o lobo ardiloso, rindo maliciosamente.'),
 Document(metadata={'Header 1': '**A Branca de Neve e o Chapeuzinho Vermelho: Uma Noite Mágica**', 'Header 2': 'Capítulo 2: Uma Missão Urgente'}, page_content='O lobo, com seus olhos brilhantes e pelagem cinzenta, sorriu maliciosamente quando Branca de Neve saiu para entregar o suco ao avô. "Não se preocupe, minha querida," disse ele, engolindo em seco. "Sou apenas um mensageiro importante."  \nBranca de Neve hesit

### 2. A Inserção e Armazenamento de Dados com o FAISS

In [2]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Load embedding model
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    model_kwargs=model_kwargs
)

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embeddings)

# Save and reload the vector store
vectorstore.save_local("faiss_index_")
persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings, allow_dangerous_deserialization=True)

# Create a retriever
retriever = persisted_vectorstore.as_retriever()

### 3. Carregando LLaMA com Ollama

In [3]:
from langchain_ollama.llms import OllamaLLM
from IPython import display

# Initialize the LLaMA model
llm = OllamaLLM(model="qwen2.5:3b")

# Test with a sample prompt
response = llm.invoke("Me conte uma piada")
display.display(display.Markdown(response))

Claro, aqui está uma piada simples para você:

Por que não fazia sentido perguntar "O que é o sol em um ambiente de querosene?"? Porque a resposta era óbvia: Não havia luz!

### 4. Orquestrando RAG com RetrievalQA

In [10]:
from langchain.chains import RetrievalQA
from IPython import display


# Create RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# Interactive query loop
while True:
    query = input("Type your query (or type 'Exit' to quit): \n")
    if query.lower() == "exit":
        break
    response = qa.invoke(query)
    display.display(display.Markdown(f"**Question:** {response["query"]}   \n**Answer:** {response["result"]}"))


**Question:** Me fale um pouco sobre a nova historia da Branca de neve   
**Answer:** A nova história da Branca de Neve que você compartilhou adicionou uma perspectiva interessante ao mito clássico, enfocando o lado do lobo e o contexto de uma vila encantada onde ela vivia junto com seu avô. Isso oferece um contraponto aos personagens conhecidos, explorando seus pontos de vista diferentes e dando mais profundidade à história.

Além disso, a introdução da amizade entre Branca de Neve e Chapeuzinho Vermelho é uma ideia original que torna a história ainda mais rica. Ela demonstra que a verdadeira coragem não vem apenas do coração dos heróis, mas também das conexões humanas.

A personagem do lobo ardiloso foi retratada de maneira inteligente: ele não é completamente vilão. Sua visão alternativa sobre Branca de Neve é revelada gradualmente à medida que os eventos se desenvolvem. Isso ajuda a criar um mundo mais complexo e rico para explorar.

Por fim, o uso da transformação da capa de Chapeuzinho Vermelho em coruja noturna como feitiço é uma ideia original e adiciona um elemento mágico às suas ações.

Em geral, essa nova história oferece um novo olhar sobre personagens e eventos conhecidos, reforçando a essência da Branca de Neve enquanto introduz elementos novos que tornam a história mais envolvente.

**Question:** Onde ela vivia e com quem?   
**Answer:** Branca de Neve vivia em uma vila encantada onde a luz do sol refletia nas casas de madeira e as flores brilhavam como diamantes sob as árvores. Ela viveu com seu avô naquela vila encantada.

**Question:** Seu avô era vivo?   
**Answer:** Na história fornecida, não há informação clara sobre se o avô de Branca de Neve estava vivo ou morto. A frase "Eu sou seu verdadeiro avô!" e a referência ao suco de morango que ela trouxe para ele sugerem que seu avô estava realmente presente na cena. No entanto, sem mais contexto específico sobre o estado do avô, não é possível responder com certeza se ele estava vivo ou morto.

**Question:** O Chapeuzinho Vermelho existe nessa historia?   
**Answer:** Não, Chapeuzinho Vermelho não existe nesta história. A história mencionada fala sobre Branca de Neve e o lobo na floresta, além do relato de uma outra aventura onde ela usa um casaco mágico para encontrar seu avô.